In [ ]:
import os
import pickle

import hera_cal

In [ ]:
# check location of hera_cal module
print(hera_cal.__file__)

In [ ]:
# checking wich branch we're working on

os.chdir('/lustre/aoc/projects/hera/mmolnar/LST_bin/hera_cal')
!git branch

In [ ]:
os.chdir('/lustre/aoc/projects/hera/mmolnar/LST_bin')

Run example lstbin script below

In [ ]:
# !zsh lstbin_run_eg.sh

## Read test no_avg files

In [ ]:
save_dir = '/lustre/aoc/projects/hera/mmolnar/LST_bin/binned_files/'
pkl_fn = os.path.join(save_dir, 'zen.test.grp1.of1.LST.1.54887.HH.OCRSL.uvh5.lst.pkl')
with open(pkl_fn, 'rb') as f:
    x = pickle.load(f)

## Reading MAD-Clip flags

In [ ]:
import lzma
import pickle
import sys
from collections import OrderedDict as odict

import pandas as pd

In [ ]:
sys.path.insert(1, '/lustre/aoc/projects/hera/mmolnar/simpleredcal')

from align_utils import idr2_jdsx

In [ ]:
clipped_flags = '/lustre/aoc/projects/hera/mmolnar/LST_bin/binned_files/lstb_mad_flags/zen.grp1.of1.LST.1.31552.bad_jds.pkl'
with open(clipped_flags, "rb") as f:
    clip_f = pickle.load(f)

In [ ]:
# Creating a odict of flags
clip_f_copy = {k: v for k, v in clip_f.items()}
new_flags = odict()
for k, v in clip_f_copy.items():
    bl_flags = odict()
    for t, f in v.items():
        # De-bin data
        bl_flags[t*2] = f[::2]
        bl_flags[t*2 + 1] = f[1::2]
    new_flags[k] = bl_flags

In [ ]:
new_flags_fn = clipped_flags.replace('zen.grp1.of1.LST', 'clipped_flags').replace('bad_jds.', '')

if not os.path.exists(new_flags_fn):
    with open(new_flags_fn, 'wb') as f:
        pickle.dump(new_flags, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Compressing the pickled file using the 
# Lempel–Ziv–Markov chain algorithm (LZMA) algorithm

new_flags_zx_fn = new_flags_fn.replace('pkl', 'xz')

if not os.path.exists(new_flags_zx_fn):
    with lzma.open(new_flags_zx_fn, 'wb') as f:
        pickle.dump(new_flags, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# # Reading back LZMA file
# with lzma.open(new_flags_zx_fn, 'rb') as file:
#     raw_data = file.read()
#     xz_flags = pickle.loads(raw_data)

## Reading LST-Binned file

In [ ]:
import numpy as np

from hera_cal.io import HERAData

from red_likelihood import group_data
from xd_utils import union_bad_ants

In [ ]:
test_uv = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/LSTBIN/one_group/grp1/zen.grp1.of1.LST.1.31552.HH.OCRSL.uvh5'
test_chans = np.arange(600, 605)
hd = HERAData(test_uv)

In [ ]:
data, flags, _ = hd.read(freq_chans=test_chans, polarizations=['ee'])

In [ ]:
_, redg, cdata = group_data(test_uv, 'ee', chans=test_chans, tints=None, bad_ants=union_bad_ants(idr2_jdsx))
flags_binned = cdata.mask

## Updating flags in lstbin.py

Code to transplant into lstbin.py. Done in the apply_xd_flags branch; not reusable yet - look into saving to UVFlag object

In [ ]:
import pickle
import os
import sys
from collections import OrderedDict as odict

import math
import numpy as np

from hera_cal.datacontainer import DataContainer
from hera_cal import io

sys.path.insert(1, '/lustre/aoc/projects/hera/mmolnar/simpleredcal')
from align_utils import idr2_jdsx
from red_utils import find_nearest, find_zen_file

In [ ]:
simpleredcal_path = '/lustre/aoc/projects/hera/mmolnar/simpleredcal'
# tot_flags_fn = 'b2f2_tot_flags.npz'
# tot_flags = np.load(os.path.join(simpleredcal_path, tot_flags_fn))['flags']

modz_fn = 'b2f2_mz_flags.npz'
modz_flags = np.load(os.path.join(simpleredcal_path, modz_fn))['simga5']
tot_flags = modz_flags

jd_label_dict = np.load(os.path.join(simpleredcal_path, 'b2f2_jd_dict.npz'), \
                        allow_pickle=True)['jd_dict'].item()

xd_dir_path = '/lustre/aoc/projects/hera/mmolnar/simpleredcal/xd_rel_dfs_nn'
with open(os.path.join(xd_dir_path, 'xd_rel_df.1.1477.ee.md.pkl'), 'rb') as f:
    md = pickle.load(f)
RedG = md['redg']

test_uv = find_zen_file(2458099.45361)
hd = io.HERAData(test_uv)

b2_freq_start = 150.3*1e6 # MHz
b2_freq_stop = 167.8*1e6 # MHz
band2_chans = np.where(np.logical_and(hd.freqs >= b2_freq_start, hd.freqs <= b2_freq_stop))[0]

with open(os.path.join(os.path.dirname(xd_dir_path), 'jd_lst_map_idr2.pkl'), 'rb') as f:
    jd_lst_map_idr2 = pickle.load(f)

jd_fracs = [int(str(round(math.modf(jdr)[0], 6))[2:7]) for jdr in jd_label_dict.keys()]
jd_int = int(list(jd_label_dict.keys())[0])
jd_lst_map_idr2 = jd_lst_map_idr2.loc[jd_int][jd_lst_map_idr2.loc[int(jd_int)].\
                      index.get_level_values('JD_frac').isin(jd_fracs)]

ref_lsts = np.concatenate(jd_lst_map_idr2['LASTs'].values).ravel()

In [ ]:
jdt = float('.'.join(os.path.basename(test_uv).split('.')[1:3]))
lstt = hd.lsts[0]

In [ ]:
# Reading the test data
data, flags, _ = hd.read()

In [ ]:
resolution = np.median(np.ediff1d(ref_lsts))

if lstt >= ref_lsts[0] - resolution/2 and lstt <= ref_lsts[-1] + resolution/2:
    lst_idx = find_nearest(ref_lsts, lstt)[1]
    day_idx = idr2_jdsx.index(int(jdt))

    flags_jdt = tot_flags[day_idx, :, lst_idx:lst_idx+60, :]

    bl_dict = [(i, j, 'ee') for i, j in RedG[:, 1:]]

    for bl in flags.keys():
        if bl in bl_dict:
            bl_idx = bl_dict.index(bl)
            flags[bl][:, band2_chans] += flags_jdt[..., bl_idx].transpose()

## LST-Bin w/ & w/o xd_rel_cal flags

In [ ]:
binned_dir = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/LSTBIN/one_group/grp1/'
lstb_f = 'zen.grp1.of1.LST.1.40949.HH.OCRSL.uvh5'
xd_binned_dir = '/lustre/aoc/projects/hera/mmolnar/LST_bin/binned_files/xdrc_flg_binned'
lstbx_f = 'zen.grp1.of1.LST.1.40949.HH.OCRSLX.uvh5' 

In [ ]:
hd_lstb = HERAData(os.path.join(binned_dir, lstb_f))
hd_lstbx = HERAData(os.path.join(xd_binned_dir, lstbx_f))

In [ ]:
data, flags, nsamples = hd_lstb.read(freq_chans=band2_chans, polarizations=['ee'])
datax, flagsx, nsamplesx = hd_lstbx.read(freq_chans=band2_chans, polarizations=['ee'])

In [ ]:
# for key in nsamples.keys():
#     eq_ns = (nsamples[key] == nsamplesx[key]).all()
#     if not eq_ns:
#         print(key)

In [ ]:
hd_lstb, redg, cmdata = group_data(os.path.join(binned_dir, lstb_f), 'ee', chans=band2_chans, \
                            tints=None, bad_ants=union_bad_ants(idr2_jdsx))
data = cmdata.data
flags = cmdata.mask

hd_lstb, redg, cmdatax = group_data(os.path.join(xd_binned_dir, lstbx_f), 'ee', chans=band2_chans, \
                            tints=None, bad_ants=union_bad_ants(idr2_jdsx))
datax = cmdatax.data
flagsx = cmdatax.mask

In [ ]:
print('{:,.0f}'.format(flags.sum()))

In [ ]:
print('{:,.0f}'.format(flagsx.sum()))

In [ ]:
nsamples_arr = np.empty_like(data, dtype=int)
for chan in range(data.shape[0]):
    nsamples_arr[chan, ...] = np.hstack([nsamples[(*bl_row[1:], 'ee')][:, chan, \
                                         np.newaxis] for bl_row in redg])
    
nsamples_arrx = np.empty_like(data, dtype=int)
for chan in range(data.shape[0]):
    nsamples_arrx[chan, ...] = np.hstack([nsamplesx[(*bl_row[1:], 'ee')][:, chan, \
                                         np.newaxis] for bl_row in redg])

In [ ]:
print('{:,.0f}'.format(nsamples_arr.sum()))

In [ ]:
print('{:,.0f}'.format(nsamples_arrx.sum()))